In [6]:
import pandas as pd
data = pd.read_csv("drug_design.csv")   # sample_data 폴더 내의 rnn_train.csv 파일을 DataFrame 타입으로 불러오기
data['SMILES'][:10000].to_csv('drug_design.txt',sep='\n')   # 윗 줄에서 불러온 파일을 구분자를 '\n'(줄 바꿈)으로 변경하여 rnn_train.txt 파일로 저장
with open('drug_design.txt', 'r') as f:   # rnn_train.txt 파일 불러오기
    text = f.read()

In [7]:
import numpy as np
# text 내의 데이터 종류를 보기위해 집합(set)으로 타입 변경 (set 타입은 데이터 중복을 없애줌)
sets=set(text)
chars = tuple(sets)
# chars 내의 데이터로 key 는 인덱스로, value 는 기존 데이터로 갖는 dicktionary를 int2char에 저장
# 나중에 모델 출력 시 숫자로 나오는데 이것을 다시 Char로 변환하기 위함
int2char = dict(enumerate(chars))
# char2int에 int2char의 key 와 value를 반전하여 저장(모델 입력에 활용 숫자 변환)
char2int = {ch: ii for ii, ch in int2char.items()}
# text의 데이터를 char2int에 key로 삽입하여 각 문자에 대응하는 숫자로 변경한다.
encoded = np.array([char2int[ch] for ch in text if ch!='\n' ])

In [8]:
encoded.shape

(184656,)

In [9]:
print(char2int)
print(int2char)
print(text[:20])
print(encoded[:20])

{'2': 0, 'I': 1, 'N': 2, '\n': 3, '1': 4, '8': 5, 'E': 6, 'L': 7, 'M': 8, '#': 9, '[': 10, '9': 11, 'c': 12, '+': 13, '4': 14, 'H': 15, '3': 16, 'l': 17, 'C': 18, ']': 19, 'r': 20, 'P': 21, '(': 22, '-': 23, 'O': 24, 'n': 25, ')': 26, '0': 27, 'F': 28, '5': 29, '=': 30, '6': 31, 'B': 32, 's': 33, 'o': 34, 'S': 35, '7': 36}
{0: '2', 1: 'I', 2: 'N', 3: '\n', 4: '1', 5: '8', 6: 'E', 7: 'L', 8: 'M', 9: '#', 10: '[', 11: '9', 12: 'c', 13: '+', 14: '4', 15: 'H', 16: '3', 17: 'l', 18: 'C', 19: ']', 20: 'r', 21: 'P', 22: '(', 23: '-', 24: 'O', 25: 'n', 26: ')', 27: '0', 28: 'F', 29: '5', 30: '=', 31: '6', 32: 'B', 33: 's', 34: 'o', 35: 'S', 36: '7'}

SMILES
0
CCO
1
C
2

[35  8  1  7  6 35 27 18 18 24  4 18  0 18 24 16  2 18 18 35]


In [10]:
def one_hot_encode(arr, n_labels):   
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)#(1600,35) 초기화
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    return one_hot

In [11]:
import torch        # 오픈소스 머신 러닝 라이브러리 pytorch
from torch import nn  # pytorch 내의 모델 정의용 라이브러리
import torch.nn.functional as F   # pytorch 내의 모델 학습 시 사용되는 함수들이 모여있는 라이브러리

In [12]:
# 데이터를 배치 사이즈에 맞게 묶어주는 함수 정의
def get_batches(arr, batch_size, seq_length): # 기본값 batch_size = 32, seq_length = 50
    # 배치 1 묶음의 길이 = 배치사이즈 * 시퀀스 길이
    batch_size_total = batch_size * seq_length
    # 배치 묶음의 갯수 지정 (전체 데이터 길이 / 배치 1 묶음의 길이)
    n_batches = len(arr)//batch_size_total
    
    # 배치 사이즈로 묶을 수 있는 크기만 남기고 나머지는 제외
    arr = arr[:n_batches * batch_size_total]
    # 배치 사이즈에 맞게 재구조화
    arr = arr.reshape((batch_size, -1)) # (32,755000)
    
    # 시퀀스 길이 단위로 n을 증가시키며 반복분 수행
    for n in range(0, arr.shape[1], seq_length): #0~755000 50단위로 짜르기 위함
        # 시퀀스 길이에 맞게 슬라이싱 (x.shape -> (32, 50))
        x = arr[:, n:n+seq_length]
        # x 와 같은 구조의 배열을 0으로 채워서 y 에 저장
        y = np.zeros_like(x)
        # x를 한칸씩 이동시켜 y에 저장
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

# GPU 사용이 가능한지 확인
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

No GPU available, training on CPU; consider making n_epochs very small.


In [13]:
batch_size = 32
seq_length = 50
batch_size_total = batch_size * seq_length #1600
print(batch_size_total)
n_batches = len(encoded)//batch_size_total
print(n_batches)

1600
115


In [14]:
arr = encoded[:n_batches * batch_size_total]
len(arr), arr.shape

(184000, (184000,))

In [15]:
arr = arr.reshape((batch_size, -1))
len(arr), arr.shape

(32, (32, 5750))

In [16]:
#x와 y의 값이 동일한데 1개 이후의 시퀀스를 예측하도록 1 shift해서 y를 만듬
for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # 한칸씩 이동 시킴
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
print(x[1])
print(len(x[1]))
print(y[1])
print(len(y[1]))
print(x.shape, y.shape)

[ 4  4 27  0  4 24 30 35  4 18 35 35 18  4  4 27  0  0  2 18  4 18 24 18
  4 30 24  4 27  0 16  2 18  4 18 18  2 18  4 30 24  4 27  0 14  2 18  4
 18 24]
50
[ 4 27  0  4 24 30 35  4 18 35 35 18  4  4 27  0  0  2 18  4 18 24 18  4
 30 24  4 27  0 16  2 18  4 18 18  2 18  4 30 24  4 27  0 14  2 18  4 18
 24  4]
50
(32, 50) (32, 50)


In [17]:
class RNN(nn.Module):
    def __init__(self, tokens, n_hidden=10, n_layers=2,
                               drop_prob=0.2, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr

        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}

        # LSTM (Long Short-Term Memory) 선언
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(n_hidden, len(self.chars))

    def forward(self, x, hidden):
        r_output, hidden = self.lstm(x, hidden)
        out = self.dropout(r_output)
        out = out.contiguous().view(-1, self.n_hidden)
        out = self.fc(out)
        return out, hidden


    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        weight = next(self.parameters()).data

        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())

        return hidden

In [18]:
def train(model, data, epochs=3, batch_size=32, seq_length=50, lr=0.001, clip=5, val_frac=0.1,print_step=1000):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    # 손실함수로는 크로스엔트로피 사용
    loss_fn = nn.CrossEntropyLoss()

    # 훈련 데이터와 검즘 데이터 분할
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]

    # GPU 사용 가능 시 net을 GPU 메모리로 이동
    if(train_on_gpu):
        model.cuda()

    # 배치 학습 수행 횟수 체크하기 위한 변수
    steps = 0
    n_chars = len(model.chars)#35
    for epoch in range(epochs):
        # hidden 레이어를 barch_size에 맞게 초기화
        h = model.init_hidden(batch_size)
        # data를 batch_size에 맞게 분할 후 하나씩 반복
        for x, y in get_batches(data, batch_size, seq_length):
            steps += 1 
            # batch_size에 맞게 나눠진 x를 one-hot 인코딩 수행
            x = one_hot_encode(x, n_chars)
            # one-hot 인코딩 된 x 를 inputs에, 텍스트 데이터인 y를 targets에 저장
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            # GPU 사용 가능 시 inputs과 targets을 GPU 메모리로 이동
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            h = tuple([each.data for each in h])
            model.zero_grad()
            output, h = model(inputs, h)
            loss = loss_fn(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()


            if steps % print_step == 0:
                val_h = model.init_hidden(batch_size)
                val_losses = []
                model.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # batch_size에 맞게 나눠진 x를 one-hot 인코딩 수행
                    x = one_hot_encode(x, n_chars)
                    # one-hot 인코딩 된 x 를 inputs에, 텍스트 데이터인 y를 targets에 저장
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    val_h = tuple([each.data for each in val_h])

                    inputs, targets = x, y
                    # GPU 사용 가능 시 inputs과 targets을 GPU 메모리로 이동
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()
                 
                    # model에 삽입하여 순전파 진행 (결과는 output과 val_h에 저장)
                    output, val_h = model(inputs, val_h)
                    val_loss = loss_fn(output, targets.view(batch_size*seq_length).long())
                    val_losses.append(val_loss.item())

                model.train()
                # 각각 Epoch, step, loss(train loss), Val Loss(val_loeese 리스트의 평균) 를 출력 
                print("Epoch: {}/{}...".format(epoch+1, epochs),
                      "Step: {}...".format(steps),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [19]:
# n_hidden, n_layer를 각각 56, 2 로 저장
n_hidden=56
n_layers=2 
model = RNN(chars, n_hidden, n_layers)
print(model) 
# 하이퍼파라미터 (batch_size, seq_length, n_epochs) 저장
batch_size = 32
seq_length = 50
epochs = 3
# 학습 시작
train(model, encoded, epochs=epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001,print_step=100)

RNN(
  (lstm): LSTM(37, 56, num_layers=2, batch_first=True, dropout=0.2)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=56, out_features=37, bias=True)
)
Epoch: 1/3... Step: 100... Loss: 2.7344... Val Loss: 2.6996
Epoch: 2/3... Step: 200... Loss: 2.5446... Val Loss: 2.5181
Epoch: 3/3... Step: 300... Loss: 2.2370... Val Loss: 2.3409


In [20]:
# 모델 이름 변수
model_name = 'rnn_1_epoch.net'


# n_hidden, n_layer, state_dict, tokens를 딕셔너리 형태로 저장
checkpoint = {'n_hidden': model.n_hidden,
              'n_layers': model.n_layers,
              'state_dict': model.state_dict(),# 학습된 모델의 가중치
              'tokens': model.chars}

# 모델을 현재 폴더에 저장
with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)


# 다음 문자 예측 방법 정의
def predict(model, char, h=None, top_k=None):

        # 입력된 문자를 해당 문자에 맞는 숫자로 저장하고(encoding) 이를 numpy array로 저장
        x = np.array([[model.char2int[char]]])
        # x 를 one-hot 인코딩 진행
        x = one_hot_encode(x, len(model.chars))
        # numpy array를 텐서로 변하여 input에 저장
        inputs = torch.from_numpy(x)

        if(train_on_gpu):
            inputs = inputs.cuda()

        h = tuple([each.data for each in h])
        out, h = model(inputs, h)

        # softmax함수를 이용하여 각 문자열일 확률 계산
        p = F.softmax(out, dim=1).data
        # 만약 p가 GPU 메모리에 있다면 메인메모리로 이동
        if(train_on_gpu):
            p = p.cpu() 

        if top_k is None:
            top_ch = np.arange(len(model.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()

        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())

        return model.int2char[char], h

In [21]:
# 문자열 예측을 위한 함수 정의
def sample(model, size, prime='The', top_k=None):
    if(train_on_gpu):
        model.cuda()
    else:
        model.cpu()
    model.eval() 
    chars = [ch for ch in prime]
    h = model.init_hidden(1)
    for ch in prime:
        char, h = predict(model, ch, h, top_k=top_k)
    chars.append(char)
    for ii in range(size):
        char, h = predict(model, chars[-1], h, top_k=top_k)
        chars.append(char)
    return ''.join(chars)

In [25]:
print(sample(model, 50, prime='C', top_k=10))

C)ONC((=N=CO()(N=C=((C(=NCNNOCN11CCC()(OO11C1C(NCC)C
